In [4]:
# Install pygame in the notebook environment (required)
%pip install pygame

import pygame
import random
from typing import List, Tuple
import math

# --- KONSTANTA (BERUBAH) ---
GRID_SIZE = 10 
CELL_SIZE = 64
MARGIN = 3 # <-- BERUBAH: Jarak antar sel
NUM_COINS = 15
FPS = 60
ANIM_DURATION_SEC = 0.15 # <-- BARU: Durasi animasi gerak (detik)

# --- WARNA (BERUBAH) ---
BG = (20, 20, 24)
CELL_BG = (40, 40, 48) # <-- BARU: Warna sel
GRID_LINE = (60, 60, 70) # (Tidak terpakai lagi, tapi disimpan)
COIN_COLOR = (240, 210, 60)
COIN_OUTLINE = (180, 150, 40) # <-- BARU
COIN_SHINE = (255, 255, 220) # <-- BARU
HUMAN_COLOR = (90, 200, 250)
AI_COLOR = (255, 100, 120)
SHADOW_COLOR = (25, 25, 28) # <-- BARU
TEXT = (235, 235, 240)
PANEL_BG = (30, 30, 38)
BLACK = (0, 0, 0)
WIN_COLORS = [
    (255, 100, 120), (90, 200, 250), (240, 210, 60), 
    (100, 255, 100), (255, 150, 255)
]

# --- UKURAN (BERUBAH) ---
# Lebar/Tinggi sekarang dihitung berdasarkan sel + margin
WIDTH = GRID_SIZE * (CELL_SIZE + MARGIN) + MARGIN
HEIGHT = GRID_SIZE * (CELL_SIZE + MARGIN) + MARGIN + 80 # Panel UI di bawah

# Posisi awal
HUMAN_START = (0, GRID_SIZE - 1)
AI_START = (GRID_SIZE - 1, 0)

Vec2 = Tuple[int, int]
Vec2f = Tuple[float, float]

# --- FUNGSI UTILITAS (BERUBAH) ---

def manhattan(a: Vec2, b: Vec2) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# Mengonversi posisi grid (0,0) ke pixel (misal 3, 3)
def grid_to_px(cell: Vec2) -> Vec2f:
    x, y = cell
    px = MARGIN + x * (CELL_SIZE + MARGIN)
    py = MARGIN + y * (CELL_SIZE + MARGIN)
    return (px, py)

# Mengonversi pixel (misal 100, 120) ke grid (misal 1, 1)
def cell_from_mouse(mx: int, my: int) -> Vec2 | None:
    if my >= HEIGHT - 80: # Di dalam panel UI
        return None
    # Rumus dibalik dari grid_to_px
    gx = (mx - MARGIN) // (CELL_SIZE + MARGIN)
    gy = (my - MARGIN) // (CELL_SIZE + MARGIN)
    
    if 0 <= gx < GRID_SIZE and 0 <= gy < GRID_SIZE:
        return (int(gx), int(gy))
    return None

def clamp(n, lo, hi):
    return max(lo, min(hi, n))

# <-- BARU: Linear Interpolation (untuk animasi)
def lerp(a: float, b: float, t: float) -> float:
    return a + (b - a) * t

def nearest_coin(from_pos: Vec2, coins: List[Vec2]) -> Vec2 | None:
    if not coins:
        return None
    best = min(coins, key=lambda c: manhattan(from_pos, c))
    return best

def step_towards(src: Vec2, dst: Vec2) -> Vec2:
    x, y = src
    dx = dst[0] - x
    dy = dst[1] - y
    if abs(dx) >= abs(dy):
        x += 1 if dx > 0 else (-1 if dx < 0 else 0)
    else:
        y += 1 if dy > 0 else (-1 if dy < 0 else 0)
    x = clamp(x, 0, GRID_SIZE - 1)
    y = clamp(y, 0, GRID_SIZE - 1)
    return (x, y)

def is_adjacent(a: Vec2, b: Vec2) -> bool:
    return manhattan(a, b) == 1

# --- KELAS GAME (DIREFAKTOR BESAR) ---
class Game:
    def __init__(self):
        self.reset()

    def reset(self):
        # Posisi grid
        self.human = HUMAN_START
        self.ai = AI_START
        
        # BARU: Posisi pixel (untuk rendering & animasi)
        self.human_px = grid_to_px(self.human)
        self.ai_px = grid_to_px(self.ai)
        self.human_start_px = self.human_px
        self.ai_start_px = self.ai_px
        self.human_target_grid = self.human
        self.ai_target_grid = self.ai
        
        self.anim_timer = 0.0 # Timer untuk animasi gerak
        
        # Logika Koin
        taken = {self.human, self.ai}
        coins = set()
        while len(coins) < NUM_COINS:
            c = (random.randrange(GRID_SIZE), random.randrange(GRID_SIZE))
            if c not in taken and c not in coins:
                coins.add(c)
        self.coins = sorted(list(coins))
        
        self.human_score = 0
        self.ai_score = 0
        
        # BARU: State machine untuk game
        # Menggantikan self.turn = "HUMAN" / "AI"
        self.state = "HUMAN_TURN" # Status: HUMAN_TURN, HUMAN_MOVING, AI_TURN, AI_MOVING, GAME_OVER
        
        self.game_over = False
        self.message = "Your turn"
        self.human_won = False

    def collect_coin_if_any(self, who: str):
        pos = self.human if who == "HUMAN" else self.ai
        if pos in self.coins:
            self.coins.remove(pos)
            if who == "HUMAN":
                self.human_score += 1
            else:
                self.ai_score += 1

    def update_game_over(self):
        if not self.coins:
            self.game_over = True
            self.state = "GAME_OVER"
            if self.human_score > self.ai_score:
                self.message = "Game Over — You win! 🎉"
                self.human_won = True
            elif self.ai_score > self.human_score:
                self.message = "Game Over — AI wins! 🤖"
            else:
                self.message = "Game Over — Draw! 🤝"

    # BERUBAH: Fungsi ini hanya *memulai* gerakan
    def human_try_move(self, target_cell: Vec2) -> bool:
        # Hanya bisa bergerak jika giliran kita dan tidak sedang animasi
        if self.state != "HUMAN_TURN":
            return False
        
        if not (0 <= target_cell[0] < GRID_SIZE and 0 <= target_cell[1] < GRID_SIZE):
            return False
            
        if is_adjacent(self.human, target_cell):
            # Atur target animasi
            self.human_start_px = self.human_px
            self.human_target_grid = target_cell
            self.anim_timer = 0.0
            self.state = "HUMAN_MOVING"
            self.message = "" # Kosongkan pesan selama bergerak
            return True
        else:
            return False

    # BERUBAH: Fungsi ini hanya *memulai* gerakan AI
    def ai_start_move(self):
        if self.state != "AI_TURN" or self.game_over:
            return
        if not self.coins:
            self.update_game_over()
            return
        
        target = nearest_coin(self.ai, self.coins)
        if target is None:
            self.update_game_over()
            return

        # Atur target animasi
        self.ai_start_px = self.ai_px
        # AI bergerak 1 langkah menuju target
        self.ai_target_grid = step_towards(self.ai, target) 
        self.anim_timer = 0.0
        self.state = "AI_MOVING"
        self.message = ""

    # BARU: Fungsi ini dipanggil setiap frame untuk update animasi
    def update_animation(self, dt_sec: float):
        if self.state == "HUMAN_MOVING":
            self.anim_timer += dt_sec
            t = self.anim_timer / ANIM_DURATION_SEC
            
            if t >= 1.0:
                # Animasi selesai
                t = 1.0
                self.human = self.human_target_grid
                self.human_px = grid_to_px(self.human)
                # Logika game (ambil koin, dll) dieksekusi SETELAH animasi selesai
                self.collect_coin_if_any("HUMAN")
                self.update_game_over()
                if not self.game_over:
                    self.state = "AI_TURN"
                    self.message = "AI thinking…"
            else:
                # Sedang animasi: Lakukan Lerp (interpolasi)
                start_px = self.human_start_px
                end_px = grid_to_px(self.human_target_grid)
                self.human_px = (
                    lerp(start_px[0], end_px[0], t),
                    lerp(start_px[1], end_px[1], t)
                )

        elif self.state == "AI_MOVING":
            self.anim_timer += dt_sec
            t = self.anim_timer / ANIM_DURATION_SEC
            
            if t >= 1.0:
                # Animasi AI selesai
                t = 1.0
                self.ai = self.ai_target_grid
                self.ai_px = grid_to_px(self.ai)
                self.collect_coin_if_any("AI")
                self.update_game_over()
                if not self.game_over:
                    self.state = "HUMAN_TURN"
                    self.message = "Your turn"
            else:
                # AI sedang animasi
                start_px = self.ai_start_px
                end_px = grid_to_px(self.ai_target_grid)
                self.ai_px = (
                    lerp(start_px[0], end_px[0], t),
                    lerp(start_px[1], end_px[1], t)
                )

# --- FUNGSI DRAW (BERUBAH) ---

# BERUBAH: Menggambar sel kotak bulat, bukan garis
def draw_grid(surface):
    surface.fill(BG) # BG sekarang jadi warna margin
    for y in range(GRID_SIZE):
        for x in range(GRID_SIZE):
            rect = pygame.Rect(
                MARGIN + x * (CELL_SIZE + MARGIN),
                MARGIN + y * (CELL_SIZE + MARGIN),
                CELL_SIZE,
                CELL_SIZE
            )
            pygame.draw.rect(surface, CELL_BG, rect, border_radius=4)

# BERUBAH: Koin lebih bagus + ada animasi bob
def draw_coins(surface, coins: List[Vec2], bob_offset: float):
    for c in coins:
        px, py = grid_to_px(c)
        cx = px + CELL_SIZE // 2
        cy = py + CELL_SIZE // 2 + bob_offset # Terapkan bob
        
        r = CELL_SIZE // 4
        
        # Gambar outline
        pygame.draw.circle(surface, COIN_OUTLINE, (cx, cy), r + 2)
        # Gambar koin
        pygame.draw.circle(surface, COIN_COLOR, (cx, cy), r)
        # Gambar kilau
        shine_r = r // 2
        pygame.draw.circle(surface, COIN_SHINE, (cx - shine_r//2, cy - shine_r//2), shine_r)


# BERUBAH: Menerima pixel (px_pos) + ada bayangan
def draw_human(surface, px_pos: Vec2f):
    px, py = px_pos
    cx = px + CELL_SIZE // 2
    cy = py + CELL_SIZE // 2
    r = CELL_SIZE // 2 - 8

    # Bayangan
    pygame.draw.circle(surface, SHADOW_COLOR, (cx, cy + 4), r)
    
    # Kepala
    pygame.draw.circle(surface, HUMAN_COLOR, (cx, cy), r)
    
    # Mata
    eye_r = CELL_SIZE // 12
    eye_y = cy - r // 3
    eye_x_off = r // 2
    pygame.draw.circle(surface, BLACK, (cx - eye_x_off, eye_y), eye_r)
    pygame.draw.circle(surface, BLACK, (cx + eye_x_off, eye_y), eye_r)

    # Senyum
    smile_y = cy + r // 3
    smile_w = r // 1.5
    pygame.draw.line(surface, BLACK, (cx - smile_w // 2, smile_y), (cx + smile_w // 2, smile_y), 4)

# BERUBAH: Menerima pixel (px_pos) + ada bayangan
def draw_ai(surface, px_pos: Vec2f):
    px, py = px_pos
    pad = CELL_SIZE // 8
    
    # Bayangan
    shadow_rect = pygame.Rect(px + pad, py + pad + 4, CELL_SIZE - 2 * pad, CELL_SIZE - 2 * pad)
    pygame.draw.rect(surface, SHADOW_COLOR, shadow_rect, border_radius=4)
    
    # Kepala (kotak)
    rect = pygame.Rect(px + pad, py + pad, CELL_SIZE - 2 * pad, CELL_SIZE - 2 * pad)
    pygame.draw.rect(surface, AI_COLOR, rect, border_radius=4)

    # Mata (kotak)
    eye_size = CELL_SIZE // 6
    eye_pad = CELL_SIZE // 6
    eye_y = py + pad + eye_pad
    eye_x1 = px + pad + eye_pad
    eye_x2 = px + CELL_SIZE - pad - eye_pad - eye_size
    pygame.draw.rect(surface, BLACK, (eye_x1, eye_y, eye_size, eye_size))
    pygame.draw.rect(surface, BLACK, (eye_x2, eye_y, eye_size, eye_size))
    
    # Mulut (garis)
    mouth_y = py + CELL_SIZE - pad - eye_pad
    pygame.draw.line(surface, BLACK, (eye_x1, mouth_y), (eye_x2 + eye_size, mouth_y), 4)

# BERUBAH: Menggunakan game.state untuk status giliran
def draw_panel(surface, font, small_font, game: Game):
    panel_y = HEIGHT - 80 # <-- Posisi panel
    panel_rect = pygame.Rect(0, panel_y, WIDTH, 80)
    pygame.draw.rect(surface, PANEL_BG, panel_rect)

    text1 = font.render(
        f"Coins left: {len(game.coins)}   Score — You: {game.human_score} | AI: {game.ai_score}",
        True, TEXT
    )
    surface.blit(text1, (16, panel_y + 10))

    # Logika status giliran berdasarkan state machine
    if game.state == "HUMAN_TURN":
        turn_txt = "Turn: YOU"
    elif game.state == "AI_TURN":
        turn_txt = "Turn: AI"
    elif game.state in ["HUMAN_MOVING", "AI_MOVING"]:
        turn_txt = "Turn: (Moving)"
    else: # GAME_OVER
        turn_txt = "Turn: —"
    
    text2 = small_font.render(
        f"{turn_txt}    Tip: Click or use WASD/Arrows.  Press R to restart.",
        True, TEXT
    )
    surface.blit(text2, (16, panel_y + 44))

    if game.message:
        msg = small_font.render(game.message, True, TEXT)
        surface.blit(msg, (WIDTH - msg.get_width() - 16, panel_y + 10))


# ... (Fungsi Konfeti tidak berubah) ...
def create_confetti(num_particles: int) -> List:
    particles = []
    for _ in range(num_particles):
        x = random.randrange(0, WIDTH)
        y = random.randrange(-HEIGHT, 0)
        dx = random.uniform(-1.5, 1.5)
        dy = random.uniform(2.0, 5.0)
        color = random.choice(WIN_COLORS)
        size = random.randint(4, 10)
        particles.append([x, y, dx, dy, color, size])
    return particles

def update_and_draw_confetti(surface, particles: List, dt: float) -> List:
    new_particles = []
    gravity = 9.8 * 20 # <-- Gravitasi sedikit disesuaikan
    
    for p in particles:
        x, y, dx, dy, color, size = p
        
        x += dx
        y += dy
        dy += gravity * dt # <-- Fisika berbasis dt
        
        if y < HEIGHT:
            new_particles.append([x, y, dx, dy, color, size])
            pygame.draw.rect(surface, color, (x, y, size, size))
            
    return new_particles

# --- FUNGSI MAIN (BERUBAH) ---
def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("=== COIN HUNT (v3) ===")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont("consolas", 22)
    small_font = pygame.font.SysFont("consolas", 18)

    game = Game()

    running = True
    ai_timer = 0
    AI_DELAY_MS = 400 
    
    confetti = []
    bob_timer = 0.0 # <-- BARU: Timer untuk animasi koin

    while running:
        dt_ms = clock.tick(FPS)
        dt_sec = dt_ms / 1000.0

        # BARU: Update timer untuk koin "bob"
        bob_timer = (bob_timer + dt_sec * 5.0) % (math.pi * 2) # * 5.0 adalah kecepatan
        bob_offset = math.sin(bob_timer) * 3.0 # * 3.0 adalah jarak (pixel)

        if game.human_won and not confetti and game.game_over:
            confetti = create_confetti(200)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_r:
                    game.reset()
                    confetti = []
                
                # BERUBAH: Cek state == "HUMAN_TURN"
                elif game.state == "HUMAN_TURN" and not game.game_over:
                    target_cell = None
                    cx, cy = game.human
                    
                    if event.key == pygame.K_w or event.key == pygame.K_UP:
                        target_cell = (cx, cy - 1)
                    elif event.key == pygame.K_s or event.key == pygame.K_DOWN:
                        target_cell = (cx, cy + 1)
                    elif event.key == pygame.K_a or event.key == pygame.K_LEFT:
                        target_cell = (cx - 1, cy)
                    elif event.key == pygame.K_d or event.key == pygame.K_RIGHT:
                        target_cell = (cx + 1, cy)
                    
                    if target_cell:
                        move_success = game.human_try_move(target_cell)
                        # Tidak perlu set ai_timer di sini, itu ditangani oleh state machine

            # BERUBAH: Cek state == "HUMAN_TURN"
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                if game.state == "HUMAN_TURN":
                    cell = cell_from_mouse(*event.pos)
                    if cell is not None:
                        move_success = game.human_try_move(cell)

        # AI move (BERUBAH: Logika timer)
        if game.state == "AI_TURN" and not game.game_over:
            ai_timer -= dt_ms
            if ai_timer <= 0:
                game.ai_start_move() # Panggil fungsi untuk *memulai* gerakan
                ai_timer = AI_DELAY_MS # Reset timer untuk gerakan AI berikutnya
        elif game.state != "AI_TURN":
             ai_timer = AI_DELAY_MS # Reset timer jika bukan giliran AI

        # BARU: Panggil update animasi setiap frame
        game.update_animation(dt_sec)

        # Render
        draw_grid(screen)
        # BERUBAH: Kirim bob_offset ke draw_coins
        draw_coins(screen, game.coins, bob_offset)
        # BERUBAH: Kirim pixel-position ke draw_human/ai
        draw_human(screen, game.human_px)
        draw_ai(screen, game.ai_px)
        draw_panel(screen, font, small_font, game)
        
        if confetti:
            # Kirim dt_sec ke confetti untuk fisika yang konsisten
            confetti = update_and_draw_confetti(screen, confetti, dt_sec)

        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
